<a href="https://colab.research.google.com/github/premswaroopmusti/Retail-Store-Location-Scraper/blob/main/Retail_Store_Location_Scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests as r
from bs4 import BeautifulSoup
import pandas as pd

In [11]:
Retail_store_list = []

for i in range(1,19):
  webpage_url = f"https://stores.vmartretail.com/?page={i}"
  req = r.get(webpage_url)
  req.text

  
  soup = BeautifulSoup(req.text,'html.parser')

  stores =  soup.find_all('div', class_ = 'col-lg-4 col-md-6 p-0 my-3')
  for i in stores:
    store_name = i.find('a',class_ = 'col-store-name txt-color-3 txt-hover-1').text.strip()
    store_address = i.find('span',class_ = 'store-address-info idx-info-card-str-add-info').text.strip()
    store_timings = i.find('span',class_ = 'store-time-info idx-info-card-str-time-info').text.strip()
    store_phone_number = i.find('a',class_ = 'txt-hover-1 txt-color-1').text.strip()

    maps_link = i.find('a',href = True, class_ = 'get_direction_click btn w-100 idx-info-card-str-dir-btn txt-color-3 bdr-color-6 bg-color-4 text-uppercase btn-hover-2' )
    Retail_store_list.append({'Store name':store_name,'Store Address': store_address, 'Store timings':store_timings, 'Store Phone Number':store_phone_number, 'Store Coordinates': maps_link['href']})

In [13]:
df = pd.DataFrame(Retail_store_list)

In [16]:
from urllib.parse import parse_qs, urlparse

url = []

for i in df['Store Coordinates']:
  url.append(i)


def remix(url):
  lat = []
  lon = []
  for i in url:
    parsed_url = urlparse(i)
    query_params = parse_qs(parsed_url.query)
    latitude, longitude = query_params['destination'][0].split(',')
    lat.append(latitude)
    lon.append(longitude)
  return lat, lon

latitudes, longitudes = remix(url)

In [20]:
df['latitude'] = latitudes
df['longitude'] = longitudes

df.to_csv('stores_with_lat_lon_new.csv', index = False)
df = pd.read_csv("/content/stores_with_lat_lon_new.csv")
df = df.drop(['Store Coordinates'], axis=1)
df.to_csv('final.csv', index = False)